Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = @__DIR__
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04 = "
data{
    int N;
    int N_actors;
    int pulled_left[N];
    int prosoc_left[N];
    int condition[N];
    int actor[N];
}
parameters{
    vector[N_actors] a;
    real bp;
    real bpC;
}
model{
    vector[N] p;
    bpC ~ normal( 0 , 10 );
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + (bp + bpC * condition[i]) * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_04",
model=m_10_04, output_format=:mcmcchains);

Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_actors" => length(unique(df[:actor])),
"actor" => df[:actor], "pulled_left" => df[:pulled_left],
"prosoc_left" => df[:prosoc_left], "condition" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
      mean   sd  5.5% 94.5% n_eff Rhat
bp    0.84 0.26  0.43  1.26  2271    1
bpC  -0.13 0.29 -0.59  0.34  2949    1

a[1] -0.74 0.27 -1.16 -0.31  3310    1
a[2] 10.88 5.20  4.57 20.73  1634    1
a[3] -1.05 0.28 -1.52 -0.59  4206    1
a[4] -1.05 0.28 -1.50 -0.60  4133    1
a[5] -0.75 0.27 -1.18 -0.32  4049    1
a[6]  0.22 0.27 -0.22  0.65  3877    1
a[7]  1.81 0.39  1.22  2.48  3807    1
";


Inference for Stan model: m_10_04_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (1.6, 1.4, 1.5, 1.4) seconds, 5.9 seconds total
Sampling took (1.3, 1.3, 1.3, 1.3) seconds, 5.3 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%  N_Eff  N_Eff/s    R_hat
lp__             -261  4.9e-02     2.1   -265   -261   -259   1855      348  1.0e+00
accept_stat__    0.85  5.9e-03    0.20   0.37   0.93    1.0   1212      228  1.0e+00
stepsize__       0.36  1.8e-02   0.025   0.32   0.38   0.39    2.0     0.38  3.4e+13
treedepth__       3.3  9.2e-02    0.62    2.0    3.0    4.0     45      8.5  1.0e+00
n_leapfrog__       12  9.6e-02     6.0    7.0     15     19   3838      720  1.0e+00
divergent__     0.064  5.3e-03    0.25   0.00   0.00    1.0   2153      404  1.0e+00
energy__          266  7.5e-02     3.0    262    266    271   1582      297  1.0e+00
a[1]            -0.74  4.6e-03    0.27   -1.2  -0.

Update sections

In [8]:
chn2 = set_section(chn, Dict(
  :parameters => ["bp", "bpC"],
  :pooled => ["a.$i" for i in 1:7],
  :internals => ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__",
    "divergent__", "energy__"]
  )
)

Object of type Chains, with data of type 1000×16×4 Array{Float64,3}

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7
internals         = lp__, accept_stat__, stepsize__, treedepth__, n_leapfrog__, divergent__, energy__
parameters        = bp, bpC

parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8427 0.2571   0.0041 0.0052 1000
bpC -0.1354 0.3027   0.0048 0.0057 1000



Describe parameter draws

In [9]:
describe(chn2)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = bp, bpC

Empirical Posterior Estimates
─────────────────────────────────────────
parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8427 0.2571   0.0041 0.0052 1000
bpC -0.1354 0.3027   0.0048 0.0057 1000

Quantiles
─────────────────────────────────────────
parameters
      2.5%   25.0%   50.0%   75.0%  97.5%
 bp -0.0191  0.6690  0.8446 1.0152 1.7708
bpC -1.1437 -0.3305 -0.1399 0.0615 0.8363



Describe pooled parameter draws

In [10]:
describe(chn2, section=:pooled)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7

Empirical Posterior Estimates
───────────────────────────────────────────
pooled
      Mean    SD   Naive SE  MCSE   ESS
a.1 -0.7392 0.2652   0.0042 0.0045 1000
a.2 10.9204 5.3025   0.0838 0.1256 1000
a.3 -1.0541 0.2725   0.0043 0.0047 1000
a.4 -1.0579 0.2773   0.0044 0.0044 1000
a.5 -0.7435 0.2691   0.0043 0.0042 1000
a.6  0.2114 0.2715   0.0043 0.0048 1000
a.7  1.8178 0.3995   0.0063 0.0063 1000

Quantiles
───────────────────────────────────────────
pooled
      2.5%   25.0%   50.0%   75.0%   97.5% 
a.1 -1.8184 -0.9220 -0.7357 -0.5594  0.3014
a.2  2.3185  6.7870  9.6885 14.1738 34.9296
a.3 -1.9935 -1.2376 -1.0500 -0.8675 -0.1238
a.4 -2.1835 -1.2415 -1.0503 -0.8662 -0.2239
a.5 -1.6829 -0.9230 -0.7327 -0.5640  0.1155
a.6 -0.7330  0.0302  0.2112  0.3963  1.3355
a.7  0.6546  1.5384  1.7943  2.0738  3.3561



Make it a DataFrame

In [11]:
df = DataFrame(chn2, [:parameters, :pooled])

,bp,bpC,a.1,a.2,a.3,a.4,a.5,a.6,a.7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.92466,0.153377,-0.656164,23.3027,-0.81731,-1.54361,-0.980074,0.103514,1.97962
2,0.933201,0.0597494,-0.686501,19.3928,-0.81704,-1.36675,-0.799793,0.261292,1.744
3,0.872626,-0.53602,-0.47967,21.2687,-0.753265,-1.41695,0.027324,0.209003,2.1808
4,0.793398,-0.213519,-0.759552,20.9193,-1.35226,-0.94371,-0.515446,0.624946,1.88272
5,1.23897,-0.356256,-0.963433,7.66638,-1.05012,-1.37931,-1.21759,-0.406136,1.53305
6,0.361049,-0.00598738,-0.210089,7.12087,-0.903534,-0.442405,-0.378081,0.608852,1.57544
7,0.432981,-0.0378694,-0.778762,6.54152,-0.762835,-0.644979,-0.238724,0.451244,2.99296
8,1.05848,-0.341068,-0.693123,6.16126,-1.11257,-1.22092,-1.26944,0.252453,2.40208
9,0.677292,-0.125061,-0.743753,7.49861,-1.39798,-0.77544,-0.428418,0.397158,2.06219


End of `10/m10.04s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*